# Compiling functions on vectors with Numba

First, [install](../index.md#installation) and import Vector and [Numba](https://numba.pydata.org/).

In [1]:
import vector
import numba as nb
import numpy as np

Numba is a just-in-time (JIT) compiler for a mathematically relevant subset of NumPy and Python. It allows you to write fast code without leaving the Python environment. The drawback of Numba is that it can only compile code blocks involving objects and functions that it recognizes.

The Vector library includes extensions to inform Numba about [vector objects](object.md), ~~[arrays of vectors](numpy.md)~~, and [arrays of Awkward Arrays](awkward.md). At the time of writing, the implementation of vector NumPy arrays is incomplete (see issue [#43](https://github.com/scikit-hep/vector/issues/43)).

Consider the following function:

In [2]:
def compute_mass(v1, v2):
    return (v1 + v2).mass

In [3]:
%timeit -n 10 compute_mass(vector.obj(px=1, py=2, pz=3, E=4), vector.obj(px=-1, py=-2, pz=-3, E=4))

The slowest run took 39.65 times longer than the fastest. This could mean that an intermediate result is being cached.
189 μs ± 386 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
@nb.njit
def compute_mass(v1, v2):
    return (v1 + v2).mass

In [5]:
%timeit -n 10 compute_mass(vector.obj(px=1, py=2, pz=3, E=4), vector.obj(px=-1, py=-2, pz=-3, E=4))

The slowest run took 1658.94 times longer than the fastest. This could mean that an intermediate result is being cached.
5.29 ms ± 12.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


When the two `MomentumObject4D` objects are passed as arguments, Numba recognizes them and replaces the Python objects with low-level structs. When it compiles the function, it recognizes `+` as the 4D `add` function and recognizes `.mass` as the `tau` component of the result.

Although this demonstrates that Numba can manipulate vector objects, there is no performance advantage (and a likely disadvantage) to compiling a calculation on just a few vectors. The `@nb.njit` result showcases this behavior, where the slowest run (the actual just-in-time compilation) is 1658.94 times longer than the fastest run.

Given that the function has now been JIT compiled once, the subsequent runs would be slightly faster than the non-JIT version runs.

In [6]:
%timeit -n 10 compute_mass(vector.obj(px=1, py=2, pz=3, E=4), vector.obj(px=-1, py=-2, pz=-3, E=4))

160 μs ± 73 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


However, the real advantage comes when many vectors are involved, in arrays.

In [7]:
# This is still not a large number. You want millions.
array = vector.Array(
    [
        [
            dict(
                {x: np.random.normal(0, 1) for x in ("px", "py", "pz")},
                E=np.random.normal(10, 1),
            )
            for inner in range(np.random.poisson(1.5))
        ]
        for outer in range(50)
    ]
)
array

<MomentumArray4D [[], [], ..., [{x: 0.633, ...}, {...}]] type='50 * var * M...'>

In [8]:
def compute_masses(array):
    out = np.empty(len(array), np.float64)
    for i, event in enumerate(array):
        total = vector.obj(px=0.0, py=0.0, pz=0.0, E=0.0)
        for vec in event:
            total = total + vec
        out[i] = total.mass
    return out

In [9]:
%timeit -n 10 compute_masses(array)

91.5 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
@nb.njit
def compute_masses(array):
    out = np.empty(len(array), np.float64)
    for i, event in enumerate(array):
        total = vector.obj(px=0.0, py=0.0, pz=0.0, E=0.0)
        for vec in event:
            total = total + vec
        out[i] = total.mass
    return out

In [11]:
%timeit -n 10 compute_masses(array)

The slowest run took 273.95 times longer than the fastest. This could mean that an intermediate result is being cached.
7.25 ms ± 17.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Again, given that the function has now been JIT compiled once, the subsequent runs would be faster (by a considerable factor) than the non-JIT version runs.

In [12]:
%timeit -n 10 compute_masses(array)

511 μs ± 222 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
